In [1]:
import sympy as sym
import numpy as np
import scipy as sp
import mpmath as mp
from functools import partial
# init_printing(use_unicode=True)

In [31]:
kx, ky, kz = sym.symbols('kx ky kz')
sigma = sym.symbols('sigma')

# kc = 1000
# p = 2
# r = 1/(1 + (sym.sqrt(kx**2 + ky**2)/kc)**p) #ADD BACK IN kz?
r=1

H = sym.Matrix([[0, -1j*sigma, 0, -1j*r, 0, 0, 0, 0, 0],
           [1j*sigma, 0, 0, 0, -1j*r, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, -1j*r, 0, 0, 0],
           [1j*r, 0, 0, 0, 0, 0, 0, kz, -ky],
           [0, 1j*r, 0, 0, 0, 0, -kz, 0, kx],
           [0, 0, 1j*r, 0, 0, 0, ky, -kx, 0],
           [0, 0, 0, 0, -kz, ky, 0, 0, 0],
           [0, 0, 0, kz, 0, -kx, 0, 0, 0],
           [0, 0, 0, -ky, kx, 0, 0, 0, 0]])

In [32]:
H

Matrix([
[          0, -1.0*I*sigma,     0, -1.0*I,      0,      0,   0,   0,   0],
[1.0*I*sigma,            0,     0,      0, -1.0*I,      0,   0,   0,   0],
[          0,            0,     0,      0,      0, -1.0*I,   0,   0,   0],
[      1.0*I,            0,     0,      0,      0,      0,   0,  kz, -ky],
[          0,        1.0*I,     0,      0,      0,      0, -kz,   0,  kx],
[          0,            0, 1.0*I,      0,      0,      0,  ky, -kx,   0],
[          0,            0,     0,      0,    -kz,     ky,   0,   0,   0],
[          0,            0,     0,     kz,      0,    -kx,   0,   0,   0],
[          0,            0,     0,    -ky,     kx,      0,   0,   0,   0]])

In [33]:
ax, ay, az, b = sym.symbols('ax ay az b')

phi, th = sym.symbols('phi th')

kx_sub = ax + b*sym.cos(phi)*sym.sin(th)
ky_sub = ay + b*sym.sin(phi)*sym.sin(th)
kz_sub = az + b*sym.cos(th)

In [34]:
H_sub = H.subs({kx: kx_sub, ky: ky_sub, kz: kz_sub})

In [35]:
H_sub

Matrix([
[          0, -1.0*I*sigma,     0,                   -1.0*I,                       0,                        0,                       0,                        0,                        0],
[1.0*I*sigma,            0,     0,                        0,                  -1.0*I,                        0,                       0,                        0,                        0],
[          0,            0,     0,                        0,                       0,                   -1.0*I,                       0,                        0,                        0],
[      1.0*I,            0,     0,                        0,                       0,                        0,                       0,           az + b*cos(th), -ay - b*sin(phi)*sin(th)],
[          0,        1.0*I,     0,                        0,                       0,                        0,         -az - b*cos(th),                        0,  ax + b*sin(th)*cos(phi)],
[          0,            0, 1.0*I,       

In [36]:
dHdkx = sym.diff(H, kx)
dHdky = sym.diff(H, ky)
dHdkz = sym.diff(H, kz)

dHdphi = sym.diff(H_sub, phi)
dHdth = sym.diff(H_sub, th)

In [37]:
dHdphi

Matrix([
[0, 0, 0,                   0,                   0,                  0,                  0,                  0,                   0],
[0, 0, 0,                   0,                   0,                  0,                  0,                  0,                   0],
[0, 0, 0,                   0,                   0,                  0,                  0,                  0,                   0],
[0, 0, 0,                   0,                   0,                  0,                  0,                  0, -b*sin(th)*cos(phi)],
[0, 0, 0,                   0,                   0,                  0,                  0,                  0, -b*sin(phi)*sin(th)],
[0, 0, 0,                   0,                   0,                  0, b*sin(th)*cos(phi), b*sin(phi)*sin(th),                   0],
[0, 0, 0,                   0,                   0, b*sin(th)*cos(phi),                  0,                  0,                   0],
[0, 0, 0,                   0,                   0, b

In [44]:
# kx_num = 1
# ky_num = 0
# kz_num = 0.4
# n = 6

def getBerryCurvature(phi_num, th_num, n, sigma_num, ax_num, ay_num, az_num, b_num):
    H_num = np.matrix(H_sub.subs({phi: phi_num, th: th_num, sigma: sigma_num, ax: ax_num, ay: ay_num, az: az_num, b: b_num}), dtype=np.complex128)
    dHdphi_num = np.matrix(dHdphi.subs({phi: phi_num, th: th_num, sigma: sigma_num, ax: ax_num, ay: ay_num, az: az_num, b: b_num}), dtype=np.complex128)
    dHdth_num = np.matrix(dHdth.subs({phi: phi_num, th: th_num, sigma: sigma_num, ax: ax_num, ay: ay_num, az: az_num, b: b_num}), dtype=np.complex128)
    evals, evecsMat = np.linalg.eigh(H_num)
    evecs = [np.matrix(evecsMat[:,i]) for i in range(len(evecsMat))]

    # Fn = 0
    # for m in range(9):
    #     if m != n:
    #         term1 = evecs[n].H @ dHdkx_num @ evecs[m]
    #         term2 = evecs[m].H @ dHdky_num @ evecs[n]
    #         term3 = evecs[m].H @ dHdkx_num @ evecs[n]
    #         term4 = evecs[n].H @ dHdky_num @ evecs[m]
    #         Fn += 1j* (term1*term2 - term3*term4)/((evals[n] - evals[m])**2)

    Fn = 0
    for m in range(9):
        if m != n:
            term1 = evecs[n].H @ dHdphi_num @ evecs[m]
            term2 = evecs[m].H @ dHdth_num @ evecs[n]
            evaldiff = evals[n] - evals[m]
            if evaldiff == 0:
                print(f"{n-4} = {evals[n]}, {m-4} = {evals[m]}")
                print(term1*term2)
                raise("Yuh")
            Fn += -2 * (term1 * term2)/((evaldiff)**2)
    Fn = Fn.imag
    
    return Fn[0,0]

In [45]:
sigma_num = 0.5
ax_num = 0
ay_num = 0
az_num = 1
b_num = 1

CList = [None] * 4
ErrorList = [None] * 4

for i in range(1,5):
    integrand = partial(getBerryCurvature, n=i+4, sigma_num=sigma_num, ax_num=ax_num, ay_num=ay_num, az_num=az_num, b_num=b_num)
    CList[i-1], ErrorList[i-1] = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
    print(f"C{i} = {CList[i-1]}")

c:\Users\decla\miniconda3\envs\gen\Lib\site-packages\scipy\integrate\_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


C1 = 6.2831853071767485
C2 = 6.283185307179586
C3 = -6.283185307179586
C4 = 3.3306690738754696e-16


In [46]:
sigma_num = 0.5
ax_num = 0
ay_num = 0
az_num = 0.75
b_num = 0.5

CList = [None] * 4
ErrorList = [None] * 4

for i in range(1,5):
    integrand = partial(getBerryCurvature, n=i+4, sigma_num=sigma_num, ax_num=ax_num, ay_num=ay_num, az_num=az_num, b_num=b_num)
    CList[i-1], ErrorList[i-1] = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
    print(f"C{i} = {CList[i-1]}")

C1 = 4.440892098500626e-16
C2 = 6.2831853071795845
C3 = -6.2831853071795845
C4 = 5.551115123125783e-17


In [23]:
def integrand(phi_num, th_num):
    n = 2
    n += 4
    return getBerryCurvature(n, phi_num, th_num)

twopiCn, error = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
Cn = twopiCn / (2*np.pi)
print(Cn)

1.0


In [24]:
def integrand(phi_num, th_num):
    n = 1
    n += 4
    return getBerryCurvature(n, phi_num, th_num)

twopiCn, error = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
Cn = twopiCn / (2*np.pi)
print(Cn)

c:\Users\decla\miniconda3\envs\gen\Lib\site-packages\scipy\integrate\_quadpack_py.py:1272: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


0.9999999999995484


In [25]:
def integrand(phi_num, th_num):
    n = 3
    n += 4
    return getBerryCurvature(n, phi_num, th_num)

twopiCn, error = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
Cn = twopiCn / (2*np.pi)
print(Cn)

-1.0


In [26]:
def integrand(phi_num, th_num):
    n = 4
    n += 4
    return getBerryCurvature(n, phi_num, th_num)

twopiCn, error = sp.integrate.dblquad(integrand, 0, np.pi, lambda th_num: 0, lambda th_num: 2*np.pi)
Cn = twopiCn / (2*np.pi)
print(Cn)

5.300924469105861e-17
